In [ ]:
from data import*
from regression import*

# %matplotlib inline
from pycaret.regression import*
from sklearn.metrics import mean_squared_error


In [ ]:
train_dataset = DATA( dataset_name='synth_00',  features=['synth_vind_1', 'synth_vind_2'], target='center_1')
test_dataset = DATA( dataset_name='arduino_02',  features=['synth_vind_1', 'synth_vind_2'], target='center_1')

In [5]:
win_size = 20
step = None

train_dataset_sg = train_dataset.segment(win_size, step=step, as_df=True)
train_dataset_sg1, train_dataset_sg2 = train_dataset_sg.split(ratio=.5)

In [6]:
exp_reg = setup(train_dataset_sg1,
                train_size = .6,
                target = 'target', 
#                 normalize = True, 
                silent = True,
                session_id = 20
                )   


Setup Succesfully Completed!


,Description,Value
0,session_id,20
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(32000, 41)"
4,Missing Values,False
5,Numeric Features,40
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [7]:
model = tune_model('rf')

IntProgress(value=0, description='Processing: ', max=16)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,13:43:28
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
ETC,. . . . . . . . . . . . . . . . . .,Calculating ETC


,MAE,MSE,RMSE,R2,RMSLE,MAPE


In [ ]:
plot_model(model, plot='error')
# plot_model(model, plot='feature')

In [ ]:
test_dataset = get_dataset( 'arduino_02',  features=['vind_1', 'vind_2'], target='center_1')
test_dataset_mtx = test_dataset.mtx(Nt_mtx='max')
np.shape(test_dataset_mtx.X)
# train_dataset_sg = train_dataset.segment(win_size, step=step, as_df=True)

In [ ]:

prediction = predict_model(model, data=validation_data_segmented[3])
smoothed_pred = signal.savgol_filter( prediction.Label, window_length=31, polyorder=1)  

ax = prediction.plot(y='target', legend=False)
prediction.plot(y='Label', ax=ax)
plt.plot(smoothed_pred)
plt.legend(['True Value', 'Prediction', 'Smoothed Prediction'])
plt.show()

In [64]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.027300,0.001400,0.037800,0.855200,0.027600,0.085600
1,CatBoost Regressor,0.028200,0.001500,0.038100,0.852400,0.028000,0.088800
2,Random Forest,0.028000,0.001500,0.038700,0.847800,0.028400,0.087800
3,Light Gradient Boosting Machine,0.028500,0.001500,0.038800,0.847000,0.028400,0.089200
4,Extreme Gradient Boosting,0.032000,0.001900,0.043600,0.807400,0.031900,0.100400
5,Gradient Boosting Regressor,0.032100,0.001900,0.043600,0.807000,0.031900,0.100600
6,K Neighbors Regressor,0.032500,0.002100,0.045800,0.787000,0.033700,0.105900
7,AdaBoost Regressor,0.043000,0.002800,0.053000,0.714900,0.039100,0.134800
8,Support Vector Machine,0.045500,0.003100,0.055500,0.688000,0.041300,0.148800
9,Decision Tree,0.039500,0.003100,0.055600,0.686000,0.040600,0.121300


In [12]:
# 